In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# 1. 加载模型
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. 加载数据集（JSONL）
dataset = load_dataset("json", data_files={"train": "./data/train.jsonl", "test": "./data/test.jsonl"})

# 3. LoRA 配置
lora_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_proj","v_proj"],
    lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model)

# 4. 训练参数
training_args = TrainingArguments(
    output_dir="./rag_lora",
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy="steps"
)

# 5. Trainer
def collate_fn(batch):
    inputs = [tokenizer(b["instruction"] + "\n" + b["input"], return_tensors="pt") for b in batch]
    labels = [tokenizer(b["output"], return_tensors="pt") for b in batch]
    # 需要自行处理padding和labels对齐
    return {"input_ids": ..., "labels": ...}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=collate_fn
)

trainer.train()
